In [34]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from typing import Optional

def query_land_registry_sparql(query: str, endpoint: str = "https://landregistry.data.gov.uk/landregistry/query", timeout: int = 120) -> pd.DataFrame:
    """
    Execute a SPARQL query against the Land Registry endpoint and return as DataFrame.
    
    Args:
        query (str): The SPARQL query string
        endpoint (str): The SPARQL endpoint URL
        
    Returns:
        pd.DataFrame: Results as a pandas DataFrame
    """
    
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(timeout)
    
    try:
        results = sparql.query().convert()
        bindings = results["results"]["bindings"]
        
        if not bindings:
            return pd.DataFrame()
        
        # Extract column names from the first result
        columns = list(bindings[0].keys())
        
        # Extract data rows
        data = []
        for binding in bindings:
            row = {}
            for col in columns:
                # Extract the actual value, handle missing optional fields
                if col in binding:
                    row[col] = binding[col]['value']
                else:
                    row[col] = None
            data.append(row)
        
        df = pd.DataFrame(data, columns=columns)
        
        # Convert data types for common columns
        if 'amount' in df.columns:
            df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            
        return df
        
    except Exception as e:
        print(f"SPARQL query failed: {e}")
        raise


postcode = "PL6 8RU"
year = 2016
query = f"""
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix lrppi: <http://landregistry.data.gov.uk/def/ppi/>
prefix lrcommon: <http://landregistry.data.gov.uk/def/common/>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?postcode ?paon ?street ?amount ?date ?locality
WHERE
{{
    # Apply date filter early - most selective constraint
    ?transx lrppi:transactionDate ?date .
    FILTER ( ?date >= "{year}-01-01"^^xsd:date )
    FILTER ( ?date <= "{year}-01-07"^^xsd:date )
    
    # Then get other transaction details
    ?transx lrppi:propertyAddress ?addr ;
            lrppi:pricePaid ?amount .
    
    # Get address components
    OPTIONAL {{?addr lrcommon:postcode ?postcode}}
    OPTIONAL {{?addr lrcommon:paon ?paon}}
    OPTIONAL {{?addr lrcommon:street ?street}}
    OPTIONAL {{?addr lrcommon:locality ?locality}}
}}
LIMIT 10000
"""


results = query_land_registry_sparql(query, "https://landregistry.data.gov.uk/landregistry/query")


In [35]:
results

,postcode,paon,street,amount,date,locality
0,L7 8SD,20,EMPRESS ROAD,120000,2016-01-01,KENSINGTON
1,BD12 8PN,12,ELIZABETH STREET,114950,2016-01-01,WYKE
2,W2 5AJ,66C,HEREFORD ROAD,620000,2016-01-01,None
3,NE64 6ED,2,VERNON PLACE,110000,2016-01-01,None
4,IP33 3HA,35,NEWMARKET ROAD,142228,2016-01-01,None
...,...,...,...,...,...,...
7566,DN22 7ZJ,2,ROSEWOOD COURT,304995,2016-01-07,None
7567,SE26 6RB,31,ORMANTON ROAD,235000,2016-01-07,None
7568,NR34 8AF,SOTHERTON HALL,KINGS LANE,500000,2016-01-07,SOTHERTON
7569,N4 2GA,"HYTHE HOUSE, 1",GREEN LANES WALK,528000,2016-01-07,None


In [36]:
7500*52

390000